import required packages

In [ ]:
import os,sys,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

load trained Sent2Vec model & predefined api_li

In [ ]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_lower_woParam_0616_768.bin') # sent2vec model
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留，注意要跟先前一致
# all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

In [ ]:
encode_dict = {}
for i,v in enumerate(api_li):
    encode_dict[v] = i+1
pickle.dump(file=open('data/tree-rep-profiles_o2o/encode_dict.pkl','wb'),obj=encode_dict) #儲存每個API function name的編碼
encode_dict
# encode_dict = pickle.load(open('data/tree-rep-profiles-partial/encode_dict.pkl','rb'))
# encode_dict

# Prepare Data vectors
* 獲取每一個資料集(train/valid/test/loner Set)execution profiles的api function name 整數對照encoding (train_api/valid_api/test_api/exp_api)
* 整個API invocation calls的Sen2Vec embedding vectors (train_emb/valid_emb/test_emb/exp_emb)
* 每一筆profile對應的路徑位置(依照順序): train_pid_paths/valid_pid_paths/test_pid_paths/exp_pid_paths

In [ ]:
def Sent2vec_emb(root_dir,max_length=226,model=model,dim=768,encode_dict=encode_dict,dil=dil,train_only=False):
    # dim = 768
    '''
    Input: 
    root_dir=>資料集目錄
    max_length=>資料集中execution profile所含API invocation calls最大長度
    model=> Sent2Vec trained model
    dim=>想要embed成的vector維度大小
    '''

    dev_pids_path = [] #儲存profile順序?
    
    sent_pad = [0]*dim
    # encode_dict = encode_dict_lower
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    hl_rep = []
    sen2vec_length_normalize = []
    sen2vec_dim_normalize = []
    loner = False
    if root_dir.split('/')[-2] in 'EXP2': #請自行修改loner的資料夾名稱!!或是如果是再跑loner的要把這個flag改成True
        loner = True
#     for rasmma in rasmma_dir:
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir +fam))[1]
        for tree in tree_dir:
#             if train_only:
#                 dest_dir = test_dir + fam + '/' + tree #r檢查此dir是否存在於TEST當中
#                 if not os.path.exists(dest_dir):
#                     continue
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl_f in hl_list:
                profile_emb = [] #整個api invocation call
                func_name_emb = [] #僅有api function name
                if not loner:
                    byterep = hl_f.replace('.profile','_byterep.pickle') #拿出hl_f所對應的byterep
                    rep = pickle.load(open(byterep,'rb')) #list type
                    if len(rep) < max_length: #Y1
                        for _ in range(max_length-len(rep)):
                            rep.append(0)
                with open(hl_f,encoding='ISO 8859-1') as f: #X2
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
                lines = re.sub('=\\n','',lines)
                lines = re.sub('y\\n','',lines)
                lines = lines.splitlines()
                if len(lines) > max_length:
                    print("ERR length too long:",hl_f,'=>',len(lines))
                    continue
                dev_pids_path.append(hl_f)
                for line in lines:
                    temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    func_name = temp.split(' ')[0] # X1
                    if func_name not in api_li:
                        print('=ERROR:=',hl_f,'=>',temp)
                    func_name_emb.append(encode_dict[func_name]) # encode証術數數字
                    emb = model.embed_sentence(temp)
                    emb = emb[0]
                    profile_emb.append(emb) #放入embdding，句子=>profile
                    sen2vec_dim_normalize.append(emb) #dim norm用
                sen2vec_length_normalize.append(len(lines))
                if len(lines) < max_length:
                    for _ in range(max_length-len(lines)):
                        profile_emb.append(sent_pad) #sent2vec vectors padding
                if not loner:
                    hl_rep.append(rep) #Y1
                all_profiles.append(profile_emb) #X2
                all_api_name.append(func_name_emb) #X1
    #             all_byterep.extend(rep_list)
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0) #X1 padding for API function name

    if not loner:
        return np.array(all_api_name) , np.array(all_profiles) , np.array(hl_rep),np.array(sen2vec_dim_normalize) ,sen2vec_length_normalize, dev_pids_path
    else:
        return np.array(all_api_name) , np.array(all_profiles) ,np.array(sen2vec_dim_normalize) ,sen2vec_length_normalize,dev_pids_path


In [ ]:
train_dir = './data/tree-rep-profiles_o2o/normal/' # Train Set位置
train_api,train_emb, train_rep, train_normalize , train_len_norm,train_pid_paths =   Sent2vec_emb(train_dir)
print(train_api.shape,train_emb.shape,train_rep.shape)
valid_dir = './data/tree-rep-profiles_o2o/DEV/' # Valid Set位置
valid_api,valid_emb, valid_rep, valid_normalize , valid_len_norm,valid_pid_paths =   Sent2vec_emb(valid_dir)
print(valid_api.shape,valid_emb.shape,valid_rep.shape)
test_dir = './data/tree-rep-profiles_o2o/TEST/' # Test Set 位置
test_api,test_emb, test_rep, test_normalize , test_len_norm,test_pid_paths =   Sent2vec_emb(test_dir)
print(test_api.shape,test_emb.shape,test_rep.shape)


In [ ]:
# 記得要先把上面function中前面的loner flag改成True
exp_dir = './data/tree-rep-profiles_o2o/EXP2/' #loner Set dir位置
exp_api,exp_emb, exp_normalize , exp_len_norm,exp_pid_paths =   Sent2vec_emb(exp_dir)
print(exp_api.shape,exp_emb.shape)

# Feature Scaling
* 僅針對Sent2Vec vectors
* Z-score normalization by each dimension

計算每個dimension的平均值跟標準差

In [ ]:
alls = np.concatenate([train_normalize,valid_normalize,test_normalize])
mean = np.mean(alls,axis=0)
std = np.std(alls,axis=0)
# mean.shape , std.shape
# 建議把mean跟std也存成pickle，方便以後可以沿用

使每個資料集的平均值=0，變異數=1

In [ ]:
def normalize_dim(emb,length,mean=mean,std=std):
    sent_emb = (emb[:length,:] - mean)/std
    final_emb = np.concatenate([sent_emb, emb[length:,:]],axis=0)
    return final_emb

In [ ]:
test_emb_norm = []
valid_emb_norm = []
train_only_emb_norm = [] #pahse2
exp_emb_norm = []
o2o_emb_norm = [] #phase1 for encoder
for emb,length in zip(train_emb,train_len_norm):
    emb_norm = normalize_dim(emb,length)
    train_only_emb_norm.append(emb_norm)
for emb,length in zip(valid_emb,valid_len_norm):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_emb,test_len_norm):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)
for emb,length in zip(exp_emb,exp_len_norm):
    emb_norm = normalize_dim(emb,length)
    exp_emb_norm.append(emb_norm)

    
train_only_emb = np.array(train_only_emb_norm)
valid_emb = np.array(valid_emb_norm)
test_emb = np.array(test_emb_norm)
exp_emb = np.array(exp_emb_norm)

print(train_only_emb.shape,valid_emb.shape,test_emb.shape,exp_emb.shape)

把最後每個資料集所得到的embedding vector以及每個profile的路徑位置存成pickle

In [ ]:
root_dir = './data/tree-rep-profiles_o2o/'
pickle.dump(file=open(root_dir + 'pids_train.pkl','wb')
            ,obj=(train_api , train_emb , train_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_valid.pkl','wb')
            ,obj=(valid_api,valid_emb,valid_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_test.pkl','wb')
            ,obj=(test_api,test_emb,test_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_exp.pkl','wb')
            ,obj=(exp_api,exp_emb),protocol=4)


pickle.dump(file=open(root_dir + 'train_pid_paths.pkl','wb'),obj=train_pid_paths)
pickle.dump(file=open(root_dir + 'valid_pid_paths.pkl','wb'),obj=valid_pid_paths)
pickle.dump(file=open(root_dir + 'test_pid_paths.pkl','wb'),obj=test_pid_paths)
pickle.dump(file=open(root_dir + 'exp_pid_paths.pkl','wb'),obj=exp_pid_paths)


***

# Appendix Spplementary
* 與任務主線無關，只是測試用的小工具
* Deprecated code. Need lots of modifications

若不想要所有家族都拿進去train，只想拿有在dev/test Set中的家族來訓練

In [ ]:
# for Encoder: train_only=True
train_dir = './data/tree-rep-profiles_o2o/normal/'
# test_dir = '' #測試資料集目錄需指定
train_only_api,train_only_emb, train_only_rep, train_only_normalize , train_only_len_norm,train_only_pid_paths =   Sent2vec_emb(train_dir,train_only=True)
print(train_only_api.shape,train_only_emb.shape,train_only_rep.shape)

針對過小過短的行為樹也進行編碼轉換

In [ ]:
# 會噴ERR是正常的
small_short_dir = './data/tree-rep-profiles_o2o/small_short/'
small_short_api,small_short_emb, small_short_rep, small_short_normalize , small_short_len_norm,small_short_pid_paths =   Sent2vec_emb(small_short_dir)
print(small_short_api.shape,small_short_emb.shape,small_short_rep.shape)

若想合併短跟小的行為樹到原本做出來的train data當中

In [ ]:
# put small_short into train data?
o2o_api = np.concatenate((train_api,small_short_api),axis=0)
o2o_emb = np.concatenate((train_emb,small_short_emb),axis=0)
o2o_rep = np.concatenate((train_rep,small_short_rep),axis=0)
o2o_normalize = np.concatenate((train_normalize,small_short_normalize),axis=0)
o2o_len_norm = train_len_norm
o2o_len_norm.extend(small_short_len_norm)
o2o_pid_paths = train_pid_paths
o2o_pid_paths.extend(small_short_pid_paths)
assert len(o2o_api) == len(o2o_emb) == len(o2o_rep) == len(o2o_pid_paths) == len(o2o_len_norm)

training o2o normalize & store

In [ ]:
for emb,length in zip(o2o_emb,o2o_len_norm):
    emb_norm = normalize_dim(emb,length)
    o2o_emb_norm.append(emb_norm)
o2o_emb = np.array(o2o_emb_norm)

pickle.dump(file=open(root_dir + 'pids_o2o.pkl','wb')
            ,obj=(o2o_api,o2o_emb,o2o_rep),protocol=4)
pickle.dump(file=open(root_dir + 'o2o_pid_paths.pkl','wb'),obj=o2o_pid_paths)